In [253]:
!pip install requests faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
103227.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


zsh:1: /Users/andriipolishko/Desktop/thesis_experiments/.venv/bin/pip: bad interpreter: /Users/andriipolishko/Desktop/thesis/experiments/.venv/bin/python: no such file or directory


In [154]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import logging
from bs4 import BeautifulSoup
import copy
import re


# Proxy rotation with Bright Data proxies

In [99]:
ucu_url = 'https://ucu.edu.ua/en/'

In [132]:
BRIGHT_DATA_USERNAME = 'brd-customer-hl_189da141-zone-datacenter_proxy1'
BRIGHT_DATA_PASSWORD = 'kybnnfzkqos0'
BRIGHT_DATA_PORT = '33335'

def get_random_proxy():
  """Returns a random proxy from the list."""

  proxies = [
    f'{BRIGHT_DATA_USERNAME}-ip-178.171.90.55:{BRIGHT_DATA_PASSWORD}@brd.superproxy.io:{BRIGHT_DATA_PORT}',
    f'{BRIGHT_DATA_USERNAME}-ip-78.138.25.40:{BRIGHT_DATA_PASSWORD}@brd.superproxy.io:{BRIGHT_DATA_PORT}',
    f'{BRIGHT_DATA_USERNAME}-ip-178.171.91.184:{BRIGHT_DATA_PASSWORD}@brd.superproxy.io:{BRIGHT_DATA_PORT}',
    f'{BRIGHT_DATA_USERNAME}-ip-103.194.114.224:{BRIGHT_DATA_PASSWORD}@brd.superproxy.io:{BRIGHT_DATA_PORT}',
  ]
  
  return random.choice(proxies)



def make_request_and_return_page_text( url):
    """Makes a request to the given URL using a random proxy. Falls back to default address if the proxy fails."""
    
    proxy = get_random_proxy()
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
      response = requests.get(url, headers=headers, proxies={"http": proxy, "https": proxy})
      if str(response.status_code).startswith('2'):
        return response.text
      else:
        # If request with proxy fails, try without proxy
        logging.error(f"Failed to fetch {url} using proxy {proxy}. Falling back to default address.")
        
        try:
          response = requests.get(url, headers=headers)
          if str(response.status_code).startswith('2'):
            return response.text
          
          else:
            logging.error(f"Failed to fetch {url} using default address.")
            
            return None
        except Exception as e:
          logging.error(f"Error fetching {url} using default address: {str(e)}")
          
          return None
    except Exception as e:
      logging.error(f"Error fetching {url} using proxy {proxy}: {str(e)}")
      
      return None

In [ ]:
def get_proxy_address(proxy):
  """Strips only the proxy address from the full proxy string."""
  if 'ip' in proxy:
    proxy = proxy.split('ip-')[1]
  if ':' in proxy:
    proxy = proxy.split(':')[0]
  return proxy

In [133]:
html = make_request_and_return_page_text(ucu_url)

In [180]:
soup = BeautifulSoup(html, "lxml")   # faster & lenient; fallback: "html.parser"


In [181]:
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<!--[if IE]>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <![endif]-->
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/manifest.json" rel="manifest"/>
<link color="#7f1716" href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon.ico" rel="shortcut icon"/>
<meta content="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/browserconfig.xml" name="msapplication-con

In [198]:
soup_text = soup.get_text(separator="\n", strip=True)
soup_text = soup_text.replace('\n', ' ')
soup_text

'Ukrainian Catholic University УКР АЇНСЬКОЮ UCU Global Admission Academics Research Campus life Towards victory About UCU Towards victory War support us Vacancies Contacts News Immortalizing the memory of the Heroes is our Christian civic duty – Collection for endowments continues During the full-scale invasion, Ukrainian Catholic University has established 10 endowments in memory of slain Heroes – students and graduates of the university who gave their lives in the fight … Continuation Archbishop Borys Gudziak Speaks at Harvard on Russia’s War Against Ukraine On Thursday, March 27, Archbishop Borys Gudziak, Metropolitan-Archbishop of the Ukrainian Catholic Archeparchy of Philadelphia, delivered a lecture at Harvard University’s Sever Hall titled “What’s at Stake in Russia’s War … Continuation Hope is born in solidarity – The University of Notre Dame opens a Ukrainian Studies Hub Religious leaders, scholars, artists, and diplomats from various countries took part in a three-day confere

In [182]:
soup_copy = copy.deepcopy(soup)
soup_copy

<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<!--[if IE]>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <![endif]-->
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/manifest.json" rel="manifest"/>
<link color="#7f1716" href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
<link href="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/favicon.ico" rel="shortcut icon"/>
<meta content="https://ucu.edu.ua/wp-content/themes/ucu/img/favicon/browserconfig.xml" name="msapplication-con

In [199]:
elements_to_remove = soup_copy(["script", "style", "noscript", 'footer'])
for tag in elements_to_remove:
    tag.decompose()

In [200]:
soup_copy

html<html lang="en-US" prefix="og: https://ogp.me/ns#"> <head>  [if IE]&gt; &lt;meta http-equiv="X-UA-Compatible" content="IE=edge"&gt; &lt;![endif]              Google Tag Manager    End Google Tag Manager   Global site tag (gtag.js) - Google Analytics     Search Engine Optimization by Rank Math - https://rankmath.com/  <title>Ukrainian Catholic University</title>                               /Rank Math WordPress SEO plugin                                     [if lt IE 9]&gt; &lt;script src="//cdnjs.cloudflare.com/ajax/libs/html5shiv/3.7.3/html5shiv.min.js"&gt;&lt;/script&gt; &lt;script src="//cdnjs.cloudflare.com/ajax/libs/respond.js/1.4.2/respond.min.js"&gt;&lt;/script&gt; &lt;![endif] </head><body class="home page-template page-template-templates page-template-home-page page-template-templateshome-page-php page page-id-2 tribe-no-js elementor-default elementor-kit-108004 elementor-page elementor-page-2">  Google Tag Manager (noscript)    End Google Tag Manager (noscript)  [if lt I

In [183]:
DROP_TAGS = [
    "script", "style", "iframe", "video", "button", "form",
    "footer", "nav", "aside", "canvas", "picture", "source"
]

In [184]:
def prune_empty_tags(soup_to_clean, keep_tags_with_attrs=("img", "a")):
    """
    Remove tags that contain no visible text **and** are not in the
    keep list (e.g. <img alt="..."> might be meaningful even if blank).

    Args:
        soup (bs4.BeautifulSoup): Parsed document.
        keep_tags_with_attrs (tuple[str]): Tag names that you want to
            preserve if they carry useful attributes (img.alt, a.href…).
    """
    for tag in soup_to_clean.find_all():
        # Skip tags we explicitly want to keep
        if tag.name in keep_tags_with_attrs:
            continue

        # Does the tag have *any* non‑whitespace text?
        if not tag.get_text(strip=True):
            tag.decompose()

In [185]:
def smart_clean(soup_to_clean):
    try:
        # 1️⃣ grab metadata, alt text, JSON‑LD *before* dropping
        metadata = {}
        for ld in soup_to_clean.select('script[type="application/ld+json"]'):
            metadata.setdefault('jsonld', []).append(ld.string)

        img_alts = [img.get('alt') for img in soup_to_clean.find_all('img') if img.get('alt')]

        # 2️⃣ drop the noisy tags
        for tag in soup_to_clean(DROP_TAGS):
            tag.decompose()
        
        # 3️⃣ optional: strip comment section by CSS selector
        for c in soup_to_clean.select("#comments, [class*='comment']"):
            c.decompose()

        return soup_to_clean, metadata, img_alts
    except Exception as e:
        logging.error(f"Error in smart_clean: {str(e)}")
        return "", {}, []

In [187]:
def preclean(soup_to_clean) -> tuple[str, dict]:
    
    # Keep <img> alt text so later splitters can pick it up
    for img in soup_to_clean.find_all("img"):
        img.replace_with(img.get("alt", "") or "")
    
    # Drop noise
    for tag in soup_to_clean(DROP_TAGS):
        tag.decompose()
    
    prune_empty_tags(soup_to_clean)

    # Normalise whitespace
    text_nodes = soup_to_clean.find_all(text=True)
    for t in text_nodes:
        t.replace_with(re.sub(r"\s+", " ", t))

    cleaned_html = str(soup_to_clean.body or soup_to_clean)          # preserve structure for LangChain
    metadata = {
        "title": (soup_to_clean.title.string if soup_to_clean.title else "").strip()
    }
    return cleaned_html, metadata

In [189]:
cleaned_html, metadata = preclean(soup_copy)
cleaned_html

/var/folders/dj/7cx50d0s24gb8_3b9ld2ywv00000gn/T/ipykernel_85344/929287989.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_nodes = soup_to_clean.find_all(text=True)


'<body class="home page-template page-template-templates page-template-home-page page-template-templateshome-page-php page page-id-2 tribe-no-js elementor-default elementor-kit-108004 elementor-page elementor-page-2">  Google Tag Manager (noscript)    End Google Tag Manager (noscript)  [if lt IE 9]&gt; &lt;p class="chromeframe"&gt;You are using an &lt;strong&gt;outdated&lt;/strong&gt; browser. Please &lt;a href="//browsehappy.com/"&gt;upgrade your browser&lt;/a&gt; to improve your experience.&lt;/p&gt; &lt;![endif] [if IE]&gt; &lt;div class="alert alert-warning"&gt; You are using an &lt;strong&gt;outdated&lt;/strong&gt; browser. Please &lt;a href="http://browsehappy.com/"&gt;upgrade your browser&lt;/a&gt; to improve your experience.&lt;/div&gt; &lt;![endif] <div class="wrapper">  start #wrap  <header class="header header--main"> <div class="header__content container-fluid"> <div class="header__top"> <div class="header__logo"> <a class="logo" href="https://ucu.edu.ua/en/" title="UCU"> U

In [162]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import HTMLSectionSplitter

def recursive_text_splitter(text_to_split):
  text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=256,
    chunk_overlap=25,
    length_function=len,
    is_separator_regex=False,
  )
  
  splitted_text = text_splitter.split_text(text_to_split)
  
  return splitted_text

def html_text_splitter(html_to_split):
  headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    # ("p", "Paragraph")
  ]

  html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

  html_header_splits = html_splitter.split_text(html_to_split)
  
  # # Remove sequential whitespaces and new lines
  # for doc in html_header_splits:
  #   doc.page_content = normalize_whitespace(doc.page_content)

  chunk_size = 500
  chunk_overlap = 30
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap
  )
  
  splitted_html = text_splitter.split_documents(html_header_splits)
  
  return splitted_html

In [ ]:
splitted_html = html_text_splitter(cleaned_html)
splitted_html

In [193]:
processed = postprocess_documents(splitted_html)

In [194]:
processed

[Document(metadata={'Header 1': '#TITLE#'}, page_content='(noscript) [if lt IE 9]> <p class="chromeframe">You are using an <strong>outdated</strong> browser. Please <a href="//browsehappy.com/">upgrade your browser</a> to improve your experience.</p> <![endif] [if IE]> <div class="alert alert-warning"> You are using an <strong>outdated</strong> browser. Please <a href="http://browsehappy.com/">upgrade your browser</a> to improve your experience.</div> <![endif] start #wrap UCU'),
 Document(metadata={'Header 3': 'News'}, page_content='News Immortalizing the memory of the Heroes is our Christian civic duty – Collection for endowments continues During the full-scale invasion, Ukrainian Catholic University has established 10 endowments in memory of slain Heroes – students and graduates of the university who gave their lives in the fight ... Continuation Archbishop Borys Gudziak Speaks at Harvard on Russia’s War Against Ukraine On Thursday, March 27, Archbishop Borys Gudziak,'),
 Document(m

In [ ]:
import xxhash

hashed_text = xxhash.xxh3_64_hexdigest(cleaned_html)

hashed_text

AttributeError: 'str' object has no attribute 'hexdigest'

In [226]:
import httpx
from httpx import AsyncHTTPTransport

proxy = f'http://{get_random_proxy()}'
transport = AsyncHTTPTransport(proxy=proxy)
client = httpx.AsyncClient(timeout=30.0, transport=transport)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Safari/537.36'}
res = await client.get(ucu_url, headers=headers)

In [263]:
aisdr_url = 'https://aisdr.com/'

aisdr_html = await client.get(aisdr_url, headers=headers)

In [234]:
ucu_html = res.text

# Context retrieval

In [247]:
def process_page_text_with_bs4_and_regex(html):
  """Processes the page text using BeautifulSoup and regex."""
  try:
    soup = BeautifulSoup(html, "lxml")
    elements_to_remove = soup(["script", "style", "noscript", 'footer'])
    for tag in elements_to_remove:
        tag.decompose()
        
    # Replace all \n with a space and combine more then 3 spaces into two
    text = soup.get_text(separator="\n", strip=True)
    text = re.sub(r'\s+', ' ', text)
    
    
    return text
  except Exception as e:
    logging.error(f"Error converting HTML to text with BeautifulSoup. Error: {str(e)}")
    
    return None

In [228]:
def recursive_text_splitter(text_to_split):
  """Splits the text into smaller chunks using RecursiveCharacterTextSplitter."""

  text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=256,
    chunk_overlap=25,
    length_function=len,
    is_separator_regex=False,
  )
  
  splitted_text = text_splitter.split_text(text_to_split)
  
  return splitted_text

In [264]:
aisdr_text = process_page_text_with_bs4_and_regex(aisdr_html)

In [265]:
splitted_text = recursive_text_splitter(aisdr_text)
splitted_text

['Book more, stress less with AI-powered sales automation - AI SDR Platform Use cases Pricing Case studies Why AiSDR? Resources Book a demo Log In Platform Use cases Pricing Case studies Why AiSDR? Resources Book a demo Log In What is AiSDR? Features Every',
 "is AiSDR? Features Every tool you need for AI sales outreach Independent AI sales assistant An extra pair of hands for your sales growth Prospecting with AI Find leads with an appetite for your offer Our best AI emails Clients' favorite emails generated",
 'emails generated by AiSDR End-to-end AI Sales Outreach All your bases covered within one solution AI for HubSpot sales Make the best of your CRM data Speak with our AI Let AiSDR try and convince you to book a meeting with us Human or AI? See if you can',
 'or AI? See if you can spot emails that were AI-generated Play the game Inbound Lead nurturing Qualification and scoring Market research Outbound AI Lead Generation Pipeline management Human or AI? See if you can spot emails 

In [ ]:
OPENAI_API_KEY=''
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"


def get_document_context(document, chunk):
  prompt = f'''
<document> 
  {document} 
</document> 
Here is the chunk we want to situate within the whole document 
<chunk> 
  {chunk} 
</chunk> 
Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else. 
'''
  try:
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "gpt-4o",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.9
    }
    
    response = requests.post(OPENAI_API_URL, headers=headers, json=data)
    
    return response.json()["choices"][0]["message"]["content"].strip()
  except Exception as e:
      print(f"Error: {e}")
      return None


In [ ]:
document_context_for_chunk = get_document_context(aisdr_text, splitted_text[0])
document_context

'The chunk is a header and navigation section from a document discussing AiSDR, an AI-powered sales automation platform designed to enhance sales outreach and lead generation, with features, pricing, case studies, and resources available.'

In [ ]:
from sentence_transformers import SentenceTransformer
sentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2')

In [250]:
def get_text_embedding(text):
  embedding = sentenceTransformer.encode(text)
    
  return embedding
    

/Users/andriipolishko/Desktop/thesis_experiments/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [257]:
import numpy as np

In [266]:
embeddings = []
for text in splitted_text:
  embedding = get_text_embedding(text)
  embeddings.append(embedding)
  
embeddings_arr = np.array([embedding for embedding in embeddings])

In [254]:
import faiss

In [267]:
d = embeddings_arr.shape[1]
sentences_index = faiss.IndexFlatL2(d)
sentences_index.add(embeddings_arr)

In [270]:
def get_closest_chunk(question, chunks, index):
  question_embeddings = np.array([get_text_embedding(question)])
  D, I = index.search(question_embeddings, k=10) # distance, index
  retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

  return retrieved_chunk



In [272]:
question = "How costly is the basic subscription?"

In [273]:
get_closest_chunk(question, splitted_text, sentences_index)

['month Fully live in <1 day First-in-class support Best outreach strategies from 50+ sales leaders under the hood Customers first (no overselling) Unlimited seats and domains 100% email deliverability 10+ daily software updates > Check out pricing Start,',
 'Check out pricing Start, then scale $900 per month 1,000 personalized emails Month-on-month payment Stop at any point Unlimited leads Unlimited seats Unlimited mailboxes and warm-up 24/7 support from GTM Engineers 150+ users love their AiSDR experience',
 'and trade show leads, nurture your inbound prospects, convert free trials into paying customers… The list is endless. Everything is handled by AI and we just select which clients should be receiving which update and the AI sends really great sales emails',
 'is < 5%. Your virtual SDR can run your SDR tech sales in any language, including English, German, Chinese, and Arabic (to name a few). Jump ahead of your competition Time kills deals. Since 78% of new business goes to the co

In [287]:
linkin_response = requests.get('https://www.linkedin.com/notifications/?filter=all')

In [289]:
linkin_response.status_code

200